In [142]:
import os
import sys
import ast
import pandas as pd

current_dir = os.getcwd()  # Current directory
pyfcs_dir = os.path.abspath(os.path.join(current_dir, '..', '..'))

# Add the PyFCS path to sys.path
sys.path.append(pyfcs_dir)

# Case 1. Human Validation Software.

Generate a dictionary with the excel that contains the results.

In [143]:
# Define the Excel file path
file_path = os.path.join(os.getcwd(), "Results", "Val_Results.xlsx")

# Load all sheets into a dictionary
sheets_dict = pd.read_excel(file_path, sheet_name=None)

# Convert each sheet into a DataFrame and store it in a dictionary
data_dict = {name: df for name, df in sheets_dict.items()}

# Display the loaded sheet names
print(f"Loaded sheets: {list(data_dict.keys())}")

Loaded sheets: ['MariaTejada']


Process human results.

In [144]:
def clean_values_confidence(values, confidence):
    """ 
    Cleans, synchronizes, and sorts Value and Confidence lists, ensuring the highest confidence value is first. 
    """
    # Convert values to lists, handling NaN cases safely
    values_list = str(values).split(",") if pd.notna(values) else []
    confidence_list = str(confidence).split(",") if pd.notna(confidence) else []

    # Trim whitespace and convert confidence to float
    values_list = [v.strip() for v in values_list]
    confidence_list = [float(c.strip()) for c in confidence_list if c.strip()]

    # Ensure both lists have the same length by padding with empty strings or zeros
    max_length = max(len(values_list), len(confidence_list))
    values_list += [""] * (max_length - len(values_list))
    confidence_list += [0.0] * (max_length - len(confidence_list))

    # Filter out empty values and confidence scores <= 0
    cleaned_pairs = [(v, c) for v, c in zip(values_list, confidence_list) if v and c > 0]

    # **Sort by confidence in descending order** (ensures highest confidence value is first)
    cleaned_pairs.sort(key=lambda x: x[1], reverse=True)

    # Separate cleaned values back into lists
    cleaned_values = [v for v, _ in cleaned_pairs]
    cleaned_confidence = [c for _, c in cleaned_pairs]

    return ", ".join(cleaned_values), ", ".join(map(str, cleaned_confidence))



# Dictionary to store processed data
dent_data = {}

for sheet_name, df in data_dict.items():
    new_rows = []

    for _, row in df.iterrows():
        tooth = row.get("Tooth", "")  # Safely get 'Tooth' column value

        # Process and clean Value/Confidence pairs
        upper_value, upper_confidence = clean_values_confidence(row.get("Upper Value"), row.get("Upper Confidence"))
        central_value, central_confidence = clean_values_confidence(row.get("Central Value"), row.get("Central Confidence"))
        lower_value, lower_confidence = clean_values_confidence(row.get("Lower Value"), row.get("Lower Confidence"))

        # Append cleaned row
        new_rows.append([
            tooth, upper_value, upper_confidence, central_value, central_confidence, lower_value, lower_confidence
        ])

    # Convert cleaned data to DataFrame
    processed_df = pd.DataFrame(new_rows, columns=[
        "Tooth", "Upper Value", "Upper Confidence", "Central Value",
        "Central Confidence", "Lower Value", "Lower Confidence"
    ])
    
    # Store processed DataFrame
    dent_data[sheet_name] = processed_df

    print(f"Processed sheet: {sheet_name}")

Processed sheet: MariaTejada


In [145]:
print(clean_values_confidence("A1, , B2", "0.9, 0.5, 0.2"))
print(clean_values_confidence(" , C3, ", "0.7, 0.6, 0.0"))
print(clean_values_confidence("D1, C1, A1", "0.8, 0.7, 0.9"))

('A1, B2', '0.9, 0.2')
('C3', '0.6')
('A1, D1, C1', '0.9, 0.8, 0.7')


In [146]:
# Increase column spacing for better readability
pd.set_option("display.colheader_justify", "center")  # Center column headers
pd.set_option("display.width", 200)  # Increase display width
pd.set_option("display.max_columns", None)  # Show all columns

for sheet_name, df in dent_data.items():
    print(f"\nSheet: {sheet_name}")
    print(df.head().to_string(index=False, col_space=20))  



Sheet: MariaTejada
       Tooth             Upper Value        Upper Confidence      Central Value      Central Confidence      Lower Value        Lower Confidence  
          A1                C1, D2              0.3, 0.2              A1, B1              0.9, 0.1              C1, B1              0.2, 0.1      
          A2                C3, C2              0.5, 0.2                  A2                   0.9                  D2                   0.3      
          A3                    D4                   0.5                  A3                   0.9                  C2                   0.5      
        A3_5                    A4                   0.2                A3_5                   0.7                  A4                   0.2      
          A4                    C4                   0.2                  A4                   0.9                  C4                   0.4      


## Load results from PyFCS (Option 1 and 2)

In [147]:
# Function to parse values from string to a list of tuples
def parse_values(cell):
    if isinstance(cell, str):
        try:
            return eval(cell)  # Converts string representation of a list into an actual list
        except (SyntaxError, NameError):
            return []  
    return []

# Function to safely parse dictionary-like strings
def parse_values_2(cell):
    if isinstance(cell, str):
        try:
            return ast.literal_eval(cell)  # Converts string representation of a dictionary into an actual dictionary
        except (ValueError, SyntaxError):
            return {}  
    return {}

# Function to process Excel files
def process_excel(file_path, parse_func, sort_dict=False):
    # Read the Excel file
    df = pd.read_excel(file_path)

    # Rename and clean the 'Imagen' column to 'Tooth'
    df.rename(columns={"Imagen": "Tooth"}, inplace=True)
    df["Tooth"] = df["Tooth"].str.replace(".png", "", regex=False)

    # Apply parsing function to relevant columns
    df["top"] = df["top"].apply(parse_func)
    df["middle"] = df["middle"].apply(parse_func)
    df["bottom"] = df["bottom"].apply(parse_func)

    # Define a function to format values based on data type
    def format_values(data):
        if isinstance(data, list):  # Handles list of tuples
            return ", ".join([str(i[0]) for i in data])
        elif isinstance(data, dict) and sort_dict:  # Handles dictionary sorting by values
            return ", ".join(sorted(data.keys(), key=data.get, reverse=True))
        return ""

    def format_confidence(data):
        if isinstance(data, list):  # Handles list of tuples
            return ", ".join([str(i[1]) for i in data])
        elif isinstance(data, dict) and sort_dict:  # Handles dictionary sorting by values
            return ", ".join(map(str, sorted(data.values(), reverse=True)))
        return ""

    # Create new formatted columns
    df["Upper Value"] = df["top"].apply(format_values)
    df["Upper Confidence"] = df["top"].apply(format_confidence)
    df["Central Value"] = df["middle"].apply(format_values)
    df["Central Confidence"] = df["middle"].apply(format_confidence)
    df["Lower Value"] = df["bottom"].apply(format_values)
    df["Lower Confidence"] = df["bottom"].apply(format_confidence)

    # Drop unnecessary columns
    df.drop(columns=["top", "middle", "bottom"], inplace=True)

    # Return the processed DataFrame with selected columns
    return df[['Tooth', 'Upper Value', 'Upper Confidence', 'Central Value', 
               'Central Confidence', 'Lower Value', 'Lower Confidence']]



# Define file paths
base_path = os.path.join(os.getcwd(), "Results", "PyFCS")
file_1 = os.path.join(base_path, "results_opt_1.xlsx")
file_2 = os.path.join(base_path, "results_opt_2.xlsx")

# Process both files
pyfcs_opt_1 = process_excel(file_1, parse_values)
pyfcs_opt_2 = process_excel(file_2, parse_values_2, sort_dict=True)

In [148]:
print(pyfcs_opt_1.head().to_string(index=False, col_space=20))
print("\n")
print(pyfcs_opt_2.head().to_string(index=False, col_space=20))

       Tooth             Upper Value        Upper Confidence      Central Value      Central Confidence      Lower Value        Lower Confidence  
          A1              A1, D2, C1       0.417, 0.334, 0.15                A1                     1.0            D2, A1            0.475, 0.306 
          A2              C3, C4, A3      0.219, 0.205, 0.179            A2, B2            0.666, 0.319        B2, D3, C4       0.4, 0.296, 0.148 
          A3              D4, C3, C4      0.331, 0.285, 0.229            A3, B2             0.83, 0.103        D4, C2, C4     0.282, 0.282, 0.227 
        A3_5                  B4, C4             0.435, 0.365      A3_5, B3, B4     0.421, 0.322, 0.245      C4, A3_5, B4     0.338, 0.272, 0.193 
          A4              C4, A4, B4      0.546, 0.279, 0.101            A4, C3            0.711, 0.225            C4, A4            0.718, 0.254 


       Tooth             Upper Value        Upper Confidence      Central Value      Central Confidence      Lower V

## Comparison of real results with cases solved by PyFCS.

In [176]:
def calculate_matches(dent_data, pyfcs_opt_1, mode=0, sheet_name=None):
    matches = {"Upper Value": 0, "Central Value": 0, "Lower Value": 0}
    value_types = ["Upper Value", "Central Value", "Lower Value"]

    # If a sheet is specified, only process that sheet
    if sheet_name:
        if sheet_name not in dent_data:
            print(f"Sheet '{sheet_name}' not found in dent_data.")
            return matches  # Return an empty dictionary if the sheet doesn't exist
        dent_data = {sheet_name: dent_data[sheet_name]}  # Filter dent_data to keep only the selected sheet

    # Iterate over the sheets (if there's only one, it will process only that)
    for sheet_name, sheet_data in dent_data.items():
        for i in range(len(sheet_data["Tooth"])):
            for value_type in value_types:
                # Extract values as sets
                dent_values = set(sheet_data[value_type][i].split(", "))
                pyfcs_values = set(pyfcs_opt_1[value_type][i].split(", "))

                if mode == 0:  # General comparison (counts all matches)
                    matches[value_type] += len(dent_values & pyfcs_values)

                elif mode == 1:  # Boolean comparison (adds 1 if there's at least one match)
                    if dent_values & pyfcs_values:
                        matches[value_type] += 1

                elif mode == 2:  # Highest confidence comparison (compares only the first value)
                    if sheet_data[value_type][i].split(", ")[0] == pyfcs_opt_1[value_type][i].split(", ")[0]:
                        matches[value_type] += 1

    return matches

### Compare to Option Summary of pixel

In [177]:
dent_index = 0
dent_key = list(dent_data.keys())[dent_index]
dent_key

'MariaTejada'

In [178]:
matches_count = calculate_matches(dent_data, pyfcs_opt_1, mode=0, sheet_name=dent_key)  # To count matches
matches_exist = calculate_matches(dent_data, pyfcs_opt_1, mode=1, sheet_name=dent_key)  # To check existence of matches
matches_highest_confidence = calculate_matches(dent_data, pyfcs_opt_1, mode=2, sheet_name=dent_key) # Highest confidence comparison

print("Matches per row:")
for col, match in matches_exist.items():
    print(f"\t{col}: {match}")

print('\n')

print("Matches with highest confidence:")
for col, match in matches_highest_confidence.items():
    print(f"\t{col}: {match}")

print('\n')
    
print("Matches in total:")
for col, match in matches_count.items():
    print(f"\t{col}: {match}")

Matches per row:
	Upper Value: 14
	Central Value: 16
	Lower Value: 12


Matches with highest confidence:
	Upper Value: 8
	Central Value: 15
	Lower Value: 9


Matches in total:
	Upper Value: 18
	Central Value: 18
	Lower Value: 13


### Compare to Option Mean of pixels

In [179]:
matches_count = calculate_matches(dent_data, pyfcs_opt_2, mode=0, sheet_name=dent_key)  # To count matches
matches_exist = calculate_matches(dent_data, pyfcs_opt_2, mode=1, sheet_name=dent_key)  # To check existence of matches
matches_highest_confidence = calculate_matches(dent_data, pyfcs_opt_2, mode=2, sheet_name=dent_key) # Highest confidence comparison

print("Matches per row:")
for col, match in matches_exist.items():
    print(f"\t{col}: {match}")

print('\n')

print("Matches with highest confidence:")
for col, match in matches_highest_confidence.items():
    print(f"\t{col}: {match}")

print('\n')
    
print("Matches in total:")
for col, match in matches_count.items():
    print(f"\t{col}: {match}")

Matches per row:
	Upper Value: 14
	Central Value: 15
	Lower Value: 8


Matches with highest confidence:
	Upper Value: 8
	Central Value: 14
	Lower Value: 2


Matches in total:
	Upper Value: 17
	Central Value: 17
	Lower Value: 8


## Time Evaluation.


In [125]:
# Define the Excel file path
file_path = os.path.join(os.getcwd(), "Results", "Val_Time.xlsx")

# Load all sheets into a dictionary
sheets_dict = pd.read_excel(file_path, sheet_name=None)

# Convert each sheet into a DataFrame and store it in a dictionary
data_dict = {name: df for name, df in sheets_dict.items()}

# Display the loaded sheet names
print(f"Loaded sheets: {list(data_dict.keys())}")

Loaded sheets: ['MariaTejada_Time']


In [126]:
top_tooth_times = {}
for key, df in data_dict.items():
    top_t = df[df['Tooth'] != 'Total'].sort_values(by='Elapsed Time (minutes)', ascending=False).head(5)
    
    # Get total time and top tooth in time
    total_time = df[df['Tooth'] == 'Total']['Elapsed Time (minutes)'].iloc[0]
    top_tooth_times[key] = {
        'total_time': total_time,
        'top_5_tooth': top_t[['Tooth', 'Elapsed Time (minutes)']].values.tolist()
    }

# Show Time Results
for key, value in top_tooth_times.items():
    print(f"Sheet: {key}")
    print(f"Total Time: {value['total_time']} minutos")
    print("Top 5 dientes con más tiempo:")
    for tooth, time in value['top_5_tooth']:
        print(f"  Diente: {tooth}, Tiempo: {time:.2f} minutos")
    print()

Sheet: MariaTejada_Time
Total Time: 52.69397112131119 minutos
Top 5 dientes con más tiempo:
  Diente: D3, Tiempo: 5.42 minutos
  Diente: D2, Tiempo: 5.25 minutos
  Diente: B3, Tiempo: 4.89 minutos
  Diente: A2, Tiempo: 4.77 minutos
  Diente: C3, Tiempo: 4.39 minutos

